In [ ]:
#get the parent folder on which this notebook is located
import os
import sys

parent_folder = os.getcwd().replace("\\notebooks","")

#add the parent folder to the system path   
if parent_folder not in sys.path:
    sys.path.append(parent_folder)

In [ ]:
os.environ["APP_CONFIGURATION_URI"] = "https://appconfig-bhy3ociqabyxc.azconfig.io"

#from dotenv import load_dotenv
from connectors import CosmosDBClient
from configuration import Configuration

config = Configuration()
cosmos_client = CosmosDBClient(config)

#load_dotenv()

In [ ]:
async def save_prompt(name:str, content:str):
    #content = content.replace("\n", "\\n")
    item = {
        "id": name,
        "system_prompt": content
    }

    await cosmos_client.upsert_document("prompts", item)

def decode_prompt(name:str):
    item = cosmos_client.get_document("prompts", name)
    if item is None:
        return None
    prompt = item.get("system_prompt", None)
    if prompt is None:
        return None
    
    prompt = prompt.replace("\\n", "\n")

    return prompt

In [ ]:
prompt = decode_prompt('document_conversion_prompt_1')
print(prompt)

In [ ]:
document_conversion_prompt_1 = """
Extract all entities, values, and field types from the provided CONTENT. Organize the extracted data into a structured JSON format that reflects the document's layout and logical grouping. The JSON structure should be flexible and based on the document type. Follow these guidelines:

   1. **Identify Sections**: Divide the document into logical sections or categories (e.g., \""Client Status,\"" \""Contact Information\"") based on field grouping.
   
   2. **Field Details**:
      - Extract each field's label as its **Name**.
      - Identify the **Type** of each field. Possible types include:
        - `TextField`: Single-line text inputs.
        - `DateField`: Date inputs.
        - `MultiSelectList`: Multiple options (e.g., checkboxes).
        - `SignatureField`: Signature inputs.
      - If applicable, specify a **DataType** (e.g., \""Text,\"" \""Currency,\"" \""Date\"") for each field.
      - If the **Type** is `MultiSelectList`, the field should include:
        - **Options**: A list of all available choices.
        - **SelectedValues**: A list of selected choices.
   
   3. **Output Format**: 
      The JSON format should look like this:
      ```json
      {
          \""Sections\"": [
              {
                  \""Name\"": \""Section Name\"",
                  \""Fields\"": [
                      { 
                          \""Name\"": \""Field Name\"", 
                          \""Type\"": \""FieldType\"", 
                          \""DataType\"": \""DataType\"",
                          \""Value\"": \""Value\"" 
                      },
                      {
                          \""Name\"": \""Income Source\"",
                          \""Type\"": \""MultiSelectList\"",
                          \""DataType\"": \""Text\"",
                          \""Options\"": [
                              \""Alimony\"", 
                              \""Rent from Tenants\"", 
                              \""Cash assistance from friends/family and relatives\"", 
                              \""Salary/wages\"",
                              \""Other options...\""
                          ],
                          \""SelectedValues\"": [
                              \""Cash assistance from friends/family and relatives\"", 
                              \""Salary/wages\""
                          ]
                      }
                  ]
              }
          ]
      }

    4. Return only the JSON structure without any additional text or explanations.

CONTENT:

{content}
"""

await save_prompt("document_conversion_prompt_1", document_conversion_prompt_1)


In [ ]:
prompt = decode_prompt('document_conversion_prompt_2')
print(prompt)

In [ ]:
document_conversion_prompt_2 = """
You are a form generator that creates a JSON document based on the provided SCHEMA and CONTENT. 

Return only the JSON structure without any additional text or explanations.

SCHEMA:

{
	"formPanels": [  //a form can have one or more form panels
		{
			"index": 0,  //the index of the panel in the form
			"panelId": ""  //the question pannel id using alphanumeric camelCase
		}
	],
	"questionPanels": [  //a form can have one or more of question panels.
		{
			"panelId": "",
            "panelHeader" : "",
    		"panelText" : "",
			"tabText": "Identification",
			"action": {
				"conditions" : [  //can be zero or more conditions based on the question id and value
					"questionId" : "existing-user",
					"value" : "no",
					"action" : "GOTO",
					"target" : "" //an existing panel id
                ],
				"next": {
					"action": "GOTO",
					"target": ""  //an existing panel id
				},
				"previous": {
					"action": "GOTO",
					"target": ""  //an existing panel id
				},
                "default" : {
					"action" : "SUBMIT",
					"target" : ""
				}
			},
            "button" : {
				"text" : "Next"
			},
			"questionSets": [ //the questions for the panel
				{
					"questionSetId": ""  #a question set id
				}
			]
		}
	],
	"questionSets": [
		{
			"questionSetId": "",  //the id of the question set using alphanumeric camelCase
			"questions": [
				{
					"questionId": "",  
					"layout": {
						"x": 0,
						"y": 0,
						"w": 6,
						"h": 1,
						"newLine": false
					},
					"meta": {
						"userDefined": false,
						"required": true,
						"cannotDelete": true,
						"reportName": "",
						"isPII": false
					},
					"input": {
						"type": "",  //can be textInput, dateInput, selectInput, phoneNumberInput, checkboxGroupInput, radioOptionsInput, postalCodeInput, textAreaInput, currencyInput, relationshipsInput
						"placeholder": "",
						"props": {},
                        "options": [
							{
								"text": "",
								"value": ""
							},
							{
								"text": "Yes",
								"value": "true"
							},
							{
								"text": "No",
								"value": "false"
							}
						]
					},
					"validations": [  //can have one or more validation based on the input type
						{
							"type": "",  //can be exists, isPhoneNumber, isNumber, isInRange, isIn, isDecimalLength, isAllIn, isEmail, isDate, isPostalCode, isCurrency, isRequired, isTextLength, isUrl
							"message": ""
                            "params" : []  //used for isAllIn, isIn validations
						}
					]
				}
			]
		}
	]
}	

CONTENT:

{content}
"""

await save_prompt("document_conversion_prompt_2", document_conversion_prompt_2)


In [ ]:
process_blob = decode_prompt('process_blob')
print(process_blob)

In [ ]:
process_blob = """
Use the 'sureimpact_file_process' function to process the file: {blob_name}
"""

await save_prompt("process_blob", process_blob)